# DATA STRUCT BLOCK
Đoạn code trên thực hiện các thao tác phân tích dữ liệu điểm thi của các thí sinh theo từng khối và xác định khối thi tốt nhất cho từng thí sinh, dựa trên các môn thi của họ.

In [ ]:
import pandas as pd
import requests
from io import StringIO
import os
from datetime import datetime, timedelta

# Load the data
cleaned_data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'
tinh_api = 'https://andyanh.id.vn/index.php/s/zbHTAjksBekNB4M/download'

def fetch_csv_from_api(api_url):
    cache_file = 'cleaned_data_cache.csv' if 'psPTAMbDrzzMnWk' in api_url else 'tinh_cache.csv'
    cache_timeout = timedelta(hours=24)
    
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")
            return pd.read_csv(cache_file)
    
    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False)
        return df
    else:
        raise Exception(f"Không thể tải dữ liệu: {response.status_code}")

# Tải dữ liệu từ API
try:
    data_df = fetch_csv_from_api(cleaned_data_api)
    tinh_df = fetch_csv_from_api(tinh_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()

# Xử lý từng khối dữ liệu để tránh tràn bộ nhớ
chunk_size = 10000
chunks = []

for chunk in pd.read_csv(StringIO(requests.get(cleaned_data_api).text), chunksize=chunk_size):
    # Merge với tinh_df
    merged_chunk = pd.merge(chunk, tinh_df, on='MaTinh', how='left')
    
    # Tính điểm trung bình các khối
    merged_chunk['Khoi_A'] = merged_chunk[['Toan', 'Ly', 'Hoa']].mean(axis=1)
    merged_chunk['Khoi_B'] = merged_chunk[['Toan', 'Sinh', 'Hoa']].mean(axis=1)
    merged_chunk['Khoi_C'] = merged_chunk[['Van', 'Lich su', 'Dia ly']].mean(axis=1)
    merged_chunk['Khoi_D'] = merged_chunk[['Toan', 'Van', 'Ngoai ngu']].mean(axis=1)

    # Kiểm tra thí sinh thi lại
    merged_chunk['Thi_Lai'] = (
        (merged_chunk[['Toan', 'Ly', 'Hoa']].notnull().sum(axis=1) == 2) |
        (merged_chunk[['Toan', 'Sinh', 'Hoa']].notnull().sum(axis=1) == 2) |
        (merged_chunk[['Van', 'Lich su', 'Dia ly']].notnull().sum(axis=1) == 2) |
        (merged_chunk[['Toan', 'Van', 'Ngoai ngu']].notnull().sum(axis=1) == 2)
    )

    # Xác định khối thi tốt nhất
    merged_chunk['Best_Khoi'] = merged_chunk[['Khoi_A', 'Khoi_B', 'Khoi_C', 'Khoi_D']].idxmax(axis=1)
    merged_chunk['Best_Khoi'] = merged_chunk.apply(
        lambda row: "Thí Sinh Thi Lại" if row['Thi_Lai'] else row['Best_Khoi'], 
        axis=1
    )

    # Chỉ giữ lại các cột cần thiết
    chunks.append(merged_chunk[['SBD', 'Best_Khoi', 'TenTinh']])

# Ghép các chunk lại với nhau
block_analysis_final_df = pd.concat(chunks, ignore_index=True)

# Tạo thư mục output nếu chưa tồn tại
os.makedirs('output', exist_ok=True)

# Xuất kết quả ra file CSV
block_analysis_final_df.to_csv('Block_Analysis_Final.csv', index=False)
